# Multi Touch Attribution

Multi-touch attribution is a method of marketing measurement that evaluates the impact of each touchpoint in driving a conversion, thereby determining the value of that specific touchpoint.
Today’s buyer journeys span many devices and touchpoints before resulting in a conversion. In order to optimize campaigns and create more customized consumer experiences, marketers need to understand which touchpoints and messages a consumer came in contact with that resulted in a positive action.

Here is a simple example of how to implement multi touch attribution models (first and last click) by using SQL (mostly) and Python. As a result we have to estimate Return on Investments (ROI) for each advertising campaign.

In [1]:
import pandas as pd
import numpy as np

#import matplotlib.pyplot as plt
#%matplotlib inline
#plt.rcParams["figure.figsize"] = (16,3)

In [2]:
import sqlite3
conn = sqlite3.connect('data_mta.db')

def get_df_from_query(query):
    return pd.read_sql(query, conn)

def create_table_from_query(query):
    return conn.execute(query)

#or use pandas
#accounts = pd.read_csv('accounts.csv', usecols=[1,2])
#costs = pd.read_csv('costs (1).csv', sep=';')
#sessions = pd.read_csv('sessions.csv', usecols=[1,2,3,4,5], dtype={'ga_cid':str}, parse_dates=[1])
#visitors = pd.read_csv('visitors.csv', usecols=[1,2,3,4], parse_dates=[3])

In [3]:
get_df_from_query("select * from sqlite_master")

,type,name,tbl_name,rootpage,sql
0,table,visitor,visitor,2,"CREATE TABLE ""visitor"" (\n""index"" INTEGER,\n ..."
1,index,ix_visitor_index,visitor,3,"CREATE INDEX ""ix_visitor_index""ON ""visitor"" (""..."
2,table,session,session,163,"CREATE TABLE ""session"" (\n""index"" INTEGER,\n ..."
3,index,ix_session_index,session,164,"CREATE INDEX ""ix_session_index""ON ""session"" (""..."
4,table,account,account,546,"CREATE TABLE ""account""(id TEXT,mrr REAL)"
5,table,cost,cost,523,"CREATE TABLE cost (\n medium text,\n source te..."
6,table,clean_data,clean_data,524,"CREATE TABLE clean_data(\n ga_cid TEXT,\n us..."


#### data model:
    
    ONE ga_cid to MANY account_id & MANY user_id
    ONE user_id to ONE ga_cid & MANY account_id
    ONE account_id to ONE user_id & ONE ga_cid

In [4]:
create_table_from_query("""drop table if exists clean_data""");

create_table_from_query("""
    create table clean_data as
    with 
    v as (
        select 
            substr(ga_cookie, 7, instr(ga_cookie, '.') + 17) ga_cid, 
            user_id,
            account_id,
            registration_date
            from visitor
            where account_id is not null),
    mta as (    
        select
            ga_cid, user_id, account_id, registration_date, event_date, medium, source, campaign, mrr
        from session
        inner join v using(ga_cid)
        left join account on account_id=id
        where registration_date >= event_date),
    mta_ as (
        select 
            *,
            row_number() over(partition by user_id order by event_date) first_click,
            row_number() over(partition by user_id order by event_date desc) last_click,
            ((julianday(registration_date) - julianday(event_date)) * 60 * 60 * 24) time_delta_sec,
            sum(mrr) over(partition by user_id) mrr_user
        from mta
    )
    select * from mta_
    """);

In [5]:
def get_mta_metrics(mta_type='first', agg_level='campaign'):
    """
    Params:
        mta_type (str): multi touch attribution model ('first' or 'last' clicks)
        agg_level (str): level of aggregation ('medium', 'source', 'campaign')
    
    Return: 
        aggregated data frame with conversion, mrr, cost, and roi
    """
    query = f"""
        with 
        c as (
            select 
                {agg_level}, sum(cost) cost
            from cost 
            group by {agg_level}),
        m as (
            select 
                {agg_level}, count(*) conversion, sum(mrr_user) mrr
            from clean_data
            where {mta_type}_click = 1
            group by {agg_level})
        select m.*, c.cost, mrr/cost roi from m
        left join c using({agg_level})
        """
    
    return get_df_from_query(query)

In [6]:
get_df_from_query("select user_id id, count(*) cnt from clean_data group by id order by cnt desc limit 6")

,id,cnt
0,a2d671cd-a9db-45b1-a661-24665109f872,3
1,46c4f287-8ea1-4950-977d-86561c05322e,2
2,4860a5e9-b5c4-496a-ad83-ff87e3770da3,2
3,a5a603be-c38c-45a9-bf6f-c18a35bdb15d,2
4,cd7d1d77-6cec-4d0c-ae2a-a2871ce4c0c4,2
5,00304ddd-6785-42ed-bd81-ee4509249833,1


In [7]:
get_mta_metrics(agg_level='campaign')

,campaign,conversion,mrr,cost,roi
0,None,809,95615.0,NaN,NaN
1,Collaboration software,53,4061.4,4000.0,1.015350
2,Competitors,121,19307.8,20000.0,0.965390
3,Free,65,6627.8,4500.0,1.472844
4,GTD,59,3981.0,14000.0,0.284357
5,Gantt software,101,12588.0,18500.0,0.680432
6,Remarketing,44,7235.2,22000.0,0.328873
7,Task software,66,4909.2,4200.0,1.168857
8,Team Management software,68,14426.4,8500.0,1.697224
9,Video collaboration software,6,0.0,14000.0,0.000000
